In [3]:
# Importar librerias
import pandas as pd
import numpy as np

import zipfile
from datetime import date
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [4]:
#Importar los datasets
url_embeddings_average_individual = zipfile.ZipFile('C:/Users/dalessam/Downloads/text_mining/Data/embeddings_average_individual.zip')
url_embeddings_sum_individual = zipfile.ZipFile('C:/Users/dalessam/Downloads/text_mining/Data/embeddings_sum_individual.zip')

embeddings_average_individual = pd.read_csv(url_embeddings_average_individual.open('embeddings_average_individual.csv'))
embeddings_sum_individual =pd.read_csv(url_embeddings_sum_individual.open('embeddings_sum_individual.csv'))

In [6]:
# Selecciono la fecha para la cual hago el corte de train y test
training_end = pd.to_datetime("2014-12-31")
num_training = len(embeddings_average_individual[pd.to_datetime(embeddings_average_individual["Date"]) <= training_end])

### Embedding Promedio

In [7]:
# Selecciono el archivo con el que se corre el modelo
data = embeddings_average_individual

In [11]:
# Se separa en train y test
x_train = data.drop([data.columns[0],"Top","Label", "Date"], axis=1)[:num_training]
x_test = data.drop([data.columns[0],"Top",'Label', 'Date'], axis=1)[num_training:]
y_train = data["Label"].values[:num_training]
y_test = data["Label"].values[num_training:]

Se cambia el shape de train y test para el input:
https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/

In [13]:
x_train_array = x_train.to_numpy()
reshape_x_train = x_train_array.reshape(40268, 1, 300)
reshape_x_train.shape

(40268, 1, 300)

In [16]:
x_test_array = x_test.to_numpy()
reshape_x_test = x_test_array.reshape(9450, 1, 300)
reshape_x_test.shape

(9450, 1, 300)

In [17]:
from numpy.testing import assert_allclose
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.
C:\Users\dalessam\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dalessam\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dalessam\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

In [20]:
n_units = 10

model = Sequential()
model.add(LSTM(n_units, input_shape=(1,300), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(n_units, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='mean_squared_error', optimizer='adam')
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# define the checkpoint
filepath="word2vec-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [21]:
# fit the model
model.fit(reshape_x_train, y_train, epochs=50, batch_size=500, callbacks=callbacks_list)
model.summary()

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
40268/40268 [==============================] - 2s 43us/step - loss: 0.6908 - accuracy: 0.5375

Epoch 00001: loss improved from inf to 0.69077, saving model to word2vec-01-0.6908.hdf5
Epoch 2/50
40268/40268 [==============================] - 1s 20us/step - loss: 0.6901 - accuracy: 0.5393

Epoch 00002: loss improved from 0.69077 to 0.69008, saving model to word2vec-02-0.6901.hdf5
Epoch 3/50
40268/40268 [==============================] - 1s 20us/step - loss: 0.6899 - accuracy: 0.5393

Epoch 00003: loss improved from 0.69008 to 0.68993, saving model to word2vec-03-0.6899.hdf5
Epoch 4/50
40268/40268 [==============================] - 1s 20us/step - loss: 0.6898 - accuracy: 0.5393

Epoch 00004: loss improved from 0.68993 to 0.68978, saving model to word2vec-04-0.6898.hdf5
Epoch 5/50
40268/40268 [==============================] - 1s 20us/step - loss: 0.6897 - accuracy: 0.5393

Epoch 00005: loss improved from 0.68978 to 0.68971, saving

Epoch 42/50
40268/40268 [==============================] - 1s 21us/step - loss: 0.6786 - accuracy: 0.5664

Epoch 00042: loss improved from 0.67905 to 0.67858, saving model to word2vec-42-0.6786.hdf5
Epoch 43/50
40268/40268 [==============================] - 1s 21us/step - loss: 0.6775 - accuracy: 0.5651

Epoch 00043: loss improved from 0.67858 to 0.67750, saving model to word2vec-43-0.6775.hdf5
Epoch 44/50
40268/40268 [==============================] - 1s 20us/step - loss: 0.6776 - accuracy: 0.5669

Epoch 00044: loss did not improve from 0.67750
Epoch 45/50
40268/40268 [==============================] - 1s 20us/step - loss: 0.6767 - accuracy: 0.5691

Epoch 00045: loss improved from 0.67750 to 0.67671, saving model to word2vec-45-0.6767.hdf5
Epoch 46/50
40268/40268 [==============================] - 1s 23us/step - loss: 0.6765 - accuracy: 0.5679

Epoch 00046: loss improved from 0.67671 to 0.67654, saving model to word2vec-46-0.6765.hdf5
Epoch 47/50
40268/40268 [=========================

In [22]:
# evaluate the model
loss, accuracy = model.evaluate(reshape_x_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy))

Accuracy: 0.513968


### Embedding Suma

In [23]:
# Selecciono el archivo con el que se corre el modelo
data = embeddings_sum_individual

In [24]:
# Se separa en train y test
x_train = data.drop([data.columns[0],"Top","Label", "Date"], axis=1)[:num_training]
x_test = data.drop([data.columns[0],"Top",'Label', 'Date'], axis=1)[num_training:]
y_train = data["Label"].values[:num_training]
y_test = data["Label"].values[num_training:]

In [25]:
x_train_array = x_train.to_numpy()
reshape_x_train = x_train_array.reshape(40268, 1, 300)
reshape_x_train.shape

(40268, 1, 300)

In [26]:
x_test_array = x_test.to_numpy()
reshape_x_test = x_test_array.reshape(9450, 1, 300)
reshape_x_test.shape

(9450, 1, 300)

In [27]:
from numpy.testing import assert_allclose
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import ModelCheckpoint

In [28]:
n_units = 10

model = Sequential()
model.add(LSTM(n_units, input_shape=(1,300), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(n_units, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='mean_squared_error', optimizer='adam')
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# define the checkpoint
filepath="word2vec-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [29]:
# fit the model
model.fit(reshape_x_train, y_train, epochs=50, batch_size=500, callbacks=callbacks_list)
model.summary()

Epoch 1/50
40268/40268 [==============================] - 2s 42us/step - loss: 0.6907 - accuracy: 0.5365

Epoch 00001: loss improved from inf to 0.69071, saving model to word2vec-01-0.6907.hdf5
Epoch 2/50
40268/40268 [==============================] - 1s 21us/step - loss: 0.6899 - accuracy: 0.5393

Epoch 00002: loss improved from 0.69071 to 0.68992, saving model to word2vec-02-0.6899.hdf5
Epoch 3/50
40268/40268 [==============================] - 1s 23us/step - loss: 0.6894 - accuracy: 0.5395

Epoch 00003: loss improved from 0.68992 to 0.68940, saving model to word2vec-03-0.6894.hdf5
Epoch 4/50
40268/40268 [==============================] - 1s 22us/step - loss: 0.6887 - accuracy: 0.5400

Epoch 00004: loss improved from 0.68940 to 0.68867, saving model to word2vec-04-0.6887.hdf5
Epoch 5/50
40268/40268 [==============================] - 1s 24us/step - loss: 0.6875 - accuracy: 0.5436

Epoch 00005: loss improved from 0.68867 to 0.68749, saving model to word2vec-05-0.6875.hdf5
Epoch 6/50
402

40268/40268 [==============================] - 1s 22us/step - loss: 0.6076 - accuracy: 0.6587

Epoch 00043: loss did not improve from 0.60667
Epoch 44/50
40268/40268 [==============================] - 1s 22us/step - loss: 0.6049 - accuracy: 0.6600

Epoch 00044: loss improved from 0.60667 to 0.60486, saving model to word2vec-44-0.6049.hdf5
Epoch 45/50
40268/40268 [==============================] - 1s 22us/step - loss: 0.6063 - accuracy: 0.6597

Epoch 00045: loss did not improve from 0.60486
Epoch 46/50
40268/40268 [==============================] - 1s 22us/step - loss: 0.6038 - accuracy: 0.6630

Epoch 00046: loss improved from 0.60486 to 0.60381, saving model to word2vec-46-0.6038.hdf5
Epoch 47/50
40268/40268 [==============================] - 1s 22us/step - loss: 0.6034 - accuracy: 0.6608

Epoch 00047: loss improved from 0.60381 to 0.60338, saving model to word2vec-47-0.6034.hdf5
Epoch 48/50
40268/40268 [==============================] - 1s 22us/step - loss: 0.6026 - accuracy: 0.6619



In [30]:
# evaluate the model
loss, accuracy = model.evaluate(reshape_x_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy))

Accuracy: 0.506878
